In [ ]:
pip install s3fs

In [ ]:
import s3fs
import pandas as pd
import tarfile

fs = s3fs.S3FileSystem(anon=False, key="KEY", secret="SECRET")

lifespans = pd.read_csv('/content/lifespans.csv', header=0)
testdf = pd.read_csv('/content/testdfclean.csv', header=0)

In [ ]:
def interpretresults(jobname,addtlname):

  fs.download(f'sagemaker-us-west-1-727244991076/{jobname}/output/output.tar.gz',f'{jobname}.tar.gz')

  with tarfile.open(f'{jobname}.tar.gz') as tar:
        tar.extractall(f'{addtlname}_results')
        tar.close()

  preds = pd.read_csv(f'/content/{addtlname}_results/predictions.csv', header=0, index_col=0)

  testdf2 = testdf.set_index('full_filename')
  testdf2 = testdf2.reindex(index=preds['Filename'])
  testdf2 = testdf2.reset_index()

  for i in range(len(preds)):
    preds.at[i,'Pred_Exact_Age'] = preds['Pred_Exact_Age'][i].replace('[','').replace(']','')
    preds.at[i,'Pred_Norm_Age'] = preds['Pred_Norm_Age'][i].replace('[','').replace(']','')
    preds.at[i,'Exact_Age_MAE'] = preds['Exact_Age_MAE'][i].replace('[','').replace(']','')
    preds.at[i,'Norm_Age_MAE'] = preds['Norm_Age_MAE'][i].replace('[','').replace(']','')

  for i in range(len(preds)):
    if preds['Exact_Age_MAE'][i] == 'nan':

      #lifespan = lifespans[preds['primaryBreed'][i]][0]
 
      lifespan = lifespans[testdf2['primaryBreed'][i]][0]
      preds.at[i,'Pred_Exact_Age'] = float(preds['Pred_Norm_Age'][i]) * lifespan
      preds.at[i,'Exact_Age_MAE'] = abs(preds['Pred_Exact_Age'][i] - preds['Exact_Age'][i])
  
  return preds

def splitdata(preds):
  oldonly = preds[preds['Norm_Age']>.5]
  oldonly['Norm_Age_MAE'] = pd.to_numeric(oldonly['Norm_Age_MAE'])
  oldonly['Exact_Age_MAE'] = pd.to_numeric(oldonly['Exact_Age_MAE'])

  youngonly = preds[preds['Norm_Age']<.5]
  youngonly['Norm_Age_MAE'] = pd.to_numeric(youngonly['Norm_Age_MAE'])
  youngonly['Exact_Age_MAE'] = pd.to_numeric(youngonly['Exact_Age_MAE'])

  return oldonly, youngonly

In [ ]:
finetunedmodel = interpretresults('VGG-Finetuned-Evaluation-2021-08-26-01-18-13-111', 'vggfinetuned')
finold, finyoung = splitdata(finetunedmodel)

In [ ]:
finold[['Norm_Age_MAE','Exact_Age_MAE']].describe()

,Norm_Age_MAE,Exact_Age_MAE
count,2.070000e+03,2070.000000
mean,3.401649e-01,4.036209
std,2.163911e-01,2.590702
min,3.576279e-07,0.000006
25%,1.788376e-01,2.104098
50%,3.225435e-01,3.779666
75%,4.656190e-01,5.520180
max,1.568055e+00,19.332592


In [ ]:
finyoung[['Norm_Age_MAE','Exact_Age_MAE']].describe()

,Norm_Age_MAE,Exact_Age_MAE
count,11775.000000,11775.000000
mean,0.082688,1.001870
std,0.092437,1.127741
min,0.000003,0.000043
25%,0.014796,0.181617
50%,0.047124,0.575266
75%,0.121265,1.465105
max,0.707121,10.746359


In [ ]:
weightermodel = interpretresults('VGG-Weighted-Evaluation-2021-08-26-18-26-11-287', 'vggweighted')
wold, wyoung = splitdata(weightermodel)

In [ ]:
wold[['Norm_Age_MAE','Exact_Age_MAE']].describe()

,Norm_Age_MAE,Exact_Age_MAE
count,2070.000000,2070.000000
mean,0.265018,3.110470
std,0.188457,2.193638
min,0.000028,0.000309
25%,0.117161,1.405257
50%,0.236035,2.795617
75%,0.380926,4.390283
max,1.446980,18.087248


In [ ]:
wyoung[['Norm_Age_MAE','Exact_Age_MAE']].describe()

,Norm_Age_MAE,Exact_Age_MAE
count,11775.000000,11775.000000
mean,0.561709,6.806785
std,0.246800,2.937719
min,0.000044,0.000555
25%,0.396504,4.912797
50%,0.529967,6.440641
75%,0.734203,8.784397
max,1.771517,19.365374


In [ ]:
xception = interpretresults('Xception-Evaluation-CustomWeight-2021-08-27-21-45-09-163', 'xception') 
xold, xyoung = splitdata(xception)

In [ ]:
xyoung[['Norm_Age_MAE','Exact_Age_MAE']].describe()

,Norm_Age_MAE,Exact_Age_MAE
count,11775.000000,11775.000000
mean,0.093857,1.134652
std,0.093887,1.142441
min,0.000013,0.000160
25%,0.018638,0.230995
50%,0.064535,0.787790
75%,0.139597,1.680140
max,0.602011,7.525139


In [ ]:
xold[['Norm_Age_MAE','Exact_Age_MAE']].describe()

,Norm_Age_MAE,Exact_Age_MAE
count,2070.000000,2070.000000
mean,0.536136,6.399382
std,0.195790,2.456390
min,0.015611,0.117085
25%,0.416552,4.797313
50%,0.512408,6.087110
75%,0.638238,7.759269
max,1.545540,19.319250


In [ ]:
xceptionraw = interpretresults('Xception-Evaluation-NotFineTuned-2021-08-26-20-28-33-963', 'xceptionraw') 
xoldraw, xyoungraw = splitdata(xceptionraw)

In [ ]:
xyoungraw[['Norm_Age_MAE','Exact_Age_MAE']].describe()

,Norm_Age_MAE,Exact_Age_MAE
count,11775.000000,11775.000000
mean,0.126461,1.530710
std,0.101716,1.223076
min,0.000041,0.000516
25%,0.042358,0.515686
50%,0.100725,1.231904
75%,0.193008,2.328781
max,0.727038,9.087975


In [ ]:
xoldraw[['Norm_Age_MAE','Exact_Age_MAE']].describe()

,Norm_Age_MAE,Exact_Age_MAE
count,2070.000000,2070.000000
mean,0.391546,4.670639
std,0.197311,2.407021
min,0.004155,0.049025
25%,0.255673,3.022923
50%,0.364844,4.303401
75%,0.495322,6.002285
max,1.556777,19.459719


In [ ]:
vuggc = interpretresults('Vgg-Evaluation-CustomWeight-2021-08-28-00-16-18-991', 'vggcustom') 
oldvuggc, youngvuggc = splitdata(vuggc)

In [ ]:
youngvuggc[['Norm_Age_MAE','Exact_Age_MAE']].describe()

,Norm_Age_MAE,Exact_Age_MAE
count,11775.000000,11775.000000
mean,0.106639,1.286728
std,0.095514,1.145421
min,0.000011,0.000140
25%,0.031549,0.390887
50%,0.079127,0.963009
75%,0.159084,1.886516
max,0.759646,9.495572


In [ ]:
oldvuggc[['Norm_Age_MAE','Exact_Age_MAE']].describe()

,Norm_Age_MAE,Exact_Age_MAE
count,2070.000000,2070.000000
mean,0.464244,5.553782
std,0.200851,2.518803
min,0.001242,0.009317
25%,0.330643,3.839372
50%,0.441514,5.284715
75%,0.572299,6.965673
max,1.628696,20.358698


In [ ]:
inception = interpretresults('Inceptionv3-Evaluation-CustomWeight-2021-08-28-00-17-51-582', 'incpeiton') 
oldincept, youngincept = splitdata(vuggc)

In [ ]:
oldincept[['Norm_Age_MAE','Exact_Age_MAE']].describe()

,Norm_Age_MAE,Exact_Age_MAE
count,2070.000000,2070.000000
mean,0.464244,5.553782
std,0.200851,2.518803
min,0.001242,0.009317
25%,0.330643,3.839372
50%,0.441514,5.284715
75%,0.572299,6.965673
max,1.628696,20.358698


In [ ]:
youngincept[['Norm_Age_MAE','Exact_Age_MAE']].describe()

,Norm_Age_MAE,Exact_Age_MAE
count,11775.000000,11775.000000
mean,0.106639,1.286728
std,0.095514,1.145421
min,0.000011,0.000140
25%,0.031549,0.390887
50%,0.079127,0.963009
75%,0.159084,1.886516
max,0.759646,9.495572
